In [2]:
import pandas as pd

In [3]:
orig_df = pd.read_csv('/home/shared/projects/sleep_analysis/Individual_sleep_activity_bout_data.csv', index_col=0)
orig_df.head()

,Channel,Condition,Light_cycle,date,time,ZT_time,Dec_ZT_time,value,sleep_counts,bout,bout_length
1,Monitor9_ch1,P384R,LD,2024-02-09,00:00:00,18:00:00,1080,1,0,1,7
8,Monitor9_ch1,P384R,LD,2024-02-09,00:07:00,18:07:00,1087,0,1,1,6
14,Monitor9_ch1,P384R,LD,2024-02-09,00:13:00,18:13:00,1093,4,0,1,12
26,Monitor9_ch1,P384R,LD,2024-02-09,00:25:00,18:25:00,1105,0,1,1,5
31,Monitor9_ch1,P384R,LD,2024-02-09,00:30:00,18:30:00,1110,2,0,1,9


In [4]:
# create list of unique Channel
channels = orig_df['Channel'].unique()
channels

array(['Monitor9_ch1', 'Monitor9_ch10', 'Monitor9_ch11', 'Monitor9_ch12',
       'Monitor9_ch13', 'Monitor9_ch14', 'Monitor9_ch15', 'Monitor9_ch16',
       'Monitor9_ch17', 'Monitor9_ch18', 'Monitor9_ch19', 'Monitor9_ch2',
       'Monitor9_ch20', 'Monitor9_ch21', 'Monitor9_ch22', 'Monitor9_ch23',
       'Monitor9_ch24', 'Monitor9_ch25', 'Monitor9_ch26', 'Monitor9_ch27',
       'Monitor9_ch28', 'Monitor9_ch29', 'Monitor9_ch3', 'Monitor9_ch30',
       'Monitor9_ch31', 'Monitor9_ch32', 'Monitor9_ch4', 'Monitor9_ch5',
       'Monitor9_ch6', 'Monitor9_ch7', 'Monitor9_ch8', 'Monitor9_ch9'],
      dtype=object)

In [5]:
# make a copy
df = orig_df.copy()

# filter sleep_count = 0
df = df[df['sleep_counts'] != 0]

# filter rows where bout_length < 5
df = df[df['bout_length'] >= 5]

# create column called 'day_or_night' where Dec_ZT_time = 0 to 720 is day, DecZT_time 721 to 1080 is night
df['day_or_night'] = df['Dec_ZT_time'].apply(lambda x: 'day' if x < 720 else 'night')

# create column bout_lengths/sleep_counts
df['bout_length_per_sleep_counts'] = df['bout_length'] / df['sleep_counts']

# create a column called 'channel_num' where Monitor9_ch1 is 1, Monitor9_ch2 is 2, etc.
df['channel_num'] = df['Channel'].apply(lambda x: int(x.split('_ch')[-1]))

df.head()

,Channel,Condition,Light_cycle,date,time,ZT_time,Dec_ZT_time,value,sleep_counts,bout,bout_length,day_or_night,bout_length_per_sleep_counts,channel_num
8,Monitor9_ch1,P384R,LD,2024-02-09,00:07:00,18:07:00,1087,0,1,1,6,night,6.0,1
26,Monitor9_ch1,P384R,LD,2024-02-09,00:25:00,18:25:00,1105,0,1,1,5,night,5.0,1
40,Monitor9_ch1,P384R,LD,2024-02-09,00:39:00,18:39:00,1119,0,1,1,16,night,16.0,1
61,Monitor9_ch1,P384R,LD,2024-02-09,01:00:00,19:00:00,1140,0,1,1,23,night,23.0,1
109,Monitor9_ch1,P384R,LD,2024-02-09,01:48:00,19:48:00,1188,0,1,1,8,night,8.0,1


In [6]:
df[df['day_or_night'] == 'day'].head(10)

,Channel,Condition,Light_cycle,date,time,ZT_time,Dec_ZT_time,value,sleep_counts,bout,bout_length,day_or_night,bout_length_per_sleep_counts,channel_num
696,Monitor9_ch1,P384R,LD,2024-02-09,11:35:00,05:35:00,335,0,1,1,8,day,8.0,1
790,Monitor9_ch1,P384R,LD,2024-02-09,13:09:00,07:09:00,429,0,1,1,10,day,10.0,1
820,Monitor9_ch1,P384R,LD,2024-02-09,13:39:00,07:39:00,459,0,1,1,5,day,5.0,1
841,Monitor9_ch1,P384R,LD,2024-02-09,14:00:00,08:00:00,480,0,1,1,13,day,13.0,1
865,Monitor9_ch1,P384R,LD,2024-02-09,14:24:00,08:24:00,504,0,1,1,12,day,12.0,1
922,Monitor9_ch1,P384R,LD,2024-02-09,15:21:00,09:21:00,561,0,1,1,5,day,5.0,1
1064,Monitor9_ch1,P384R,LD,2024-02-09,17:43:00,11:43:00,703,0,1,1,5,day,5.0,1
2098,Monitor9_ch1,P384R,LD,2024-02-10,10:57:00,04:57:00,297,0,1,1,6,day,6.0,1
2287,Monitor9_ch1,P384R,LD,2024-02-10,14:06:00,08:06:00,486,0,1,1,7,day,7.0,1
2300,Monitor9_ch1,P384R,LD,2024-02-10,14:19:00,08:19:00,499,0,1,1,7,day,7.0,1


In [7]:
# group by time_of_day, channel, date and then sum and average bout and bout_length
grouped = df.groupby(['day_or_night', 'Channel', 'channel_num', 'Condition', 'date']).agg({'bout': ['sum'], 'bout_length': ['sum', 'mean']}).reset_index()

# sort grouped by channel_num and date
grouped = grouped.sort_values(['day_or_night','channel_num', 'date'])

# format bout_length mean to 2 decimal places
grouped['bout_length', 'mean'] = grouped['bout_length', 'mean'].apply(lambda x: round(x, 2))

# transform the multi-index columns to single index
grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]

# replace columns that end with '_' with ''
grouped.columns = grouped.columns.str.replace('_$', '', regex=True)

grouped.head(5)

,day_or_night,Channel,channel_num,Condition,date,bout_sum,bout_length_sum,bout_length_mean
0,day,Monitor9_ch1,1,P384R,2024-02-09,7,58,8.29
1,day,Monitor9_ch1,1,P384R,2024-02-10,10,115,11.50
2,day,Monitor9_ch1,1,P384R,2024-02-11,11,86,7.82
26,day,Monitor9_ch2,2,P384R,2024-02-09,8,78,9.75
27,day,Monitor9_ch2,2,P384R,2024-02-10,11,95,8.64


In [8]:
# create a new grouped dataframe that is grouped by day_or_night, channel_num and averages bout sum and bout_length mean
grouped2 = grouped.groupby(['day_or_night', 'Channel', 'channel_num', 'Condition']).agg({'bout_sum': ['mean'], 'bout_length_mean': ['mean']}).reset_index()

# sort by channel_num
grouped2 = grouped2.sort_values(['day_or_night', 'channel_num'])

grouped2.head()


,day_or_night,Channel,channel_num,Condition,bout_sum,bout_length_mean
,,,,,mean,mean
0,day,Monitor9_ch1,1,P384R,9.333333,9.203333
11,day,Monitor9_ch2,2,P384R,9.666667,9.196667
22,day,Monitor9_ch3,3,P384R,4.333333,5.930000
26,day,Monitor9_ch4,4,P384R,2.333333,16.600000
27,day,Monitor9_ch5,5,P384R,5.666667,9.746667


In [20]:
for time_of_day in ['day', 'night']:
    #print(time_of_day)
    
    for channel in channels:
        
        for date in df['Date'].unique():
            # get the sleep count per channel_num
            sleep_count_total = df[(df['day_or_night'] == time_of_day) & (df['Channel'] == channel)]['bout'].sum()
            sleep_count_mean = df[(df['day_or_night'] == time_of_day) & (df['Channel'] == channel)]['bout'].mean()
            bout_length_total = df[(df['day_or_night'] == time_of_day) & (df['Channel'] == channel)]['bout_length'].sum()
            # get only 2 decimal points for bout_length_mean
            bout_length_mean = df[(df['day_or_night'] == time_of_day) & (df['Channel'] == channel)]['bout_length'].mean()
            bout_length_mean = f"{bout_length_mean:.2f}"
            print(f"{time_of_day}\t{channel}\t{sleep_count_total}\t{sleep_count_mean}\t{bout_length_total}\t{bout_length_mean}")

day	Monitor9_ch1	28	1.0	259	9.25
day	Monitor9_ch10	13	1.0	107	8.23
day	Monitor9_ch11	1	1.0	5	5.00
day	Monitor9_ch12	5	1.0	63	12.60
day	Monitor9_ch13	10	1.0	80	8.00
day	Monitor9_ch14	17	1.0	703	41.35
day	Monitor9_ch15	5	1.0	413	82.60
day	Monitor9_ch16	15	1.0	114	7.60
day	Monitor9_ch17	19	1.0	137	7.21
day	Monitor9_ch18	16	1.0	170	10.62
day	Monitor9_ch19	28	1.0	323	11.54
day	Monitor9_ch2	29	1.0	265	9.14
day	Monitor9_ch20	20	1.0	245	12.25
day	Monitor9_ch21	7	1.0	506	72.29
day	Monitor9_ch22	19	1.0	536	28.21
day	Monitor9_ch23	4	1.0	35	8.75
day	Monitor9_ch24	13	1.0	206	15.85
day	Monitor9_ch25	13	1.0	121	9.31
day	Monitor9_ch26	22	1.0	274	12.45
day	Monitor9_ch27	15	1.0	594	39.60
day	Monitor9_ch28	4	1.0	24	6.00
day	Monitor9_ch29	6	1.0	44	7.33
day	Monitor9_ch3	13	1.0	84	6.46
day	Monitor9_ch30	27	1.0	223	8.26
day	Monitor9_ch31	9	1.0	282	31.33
day	Monitor9_ch32	9	1.0	68	7.56
day	Monitor9_ch4	7	1.0	145	20.71
day	Monitor9_ch5	17	1.0	164	9.65
day	Monitor9_ch6	2	1.0	375	187.50
day	Monitor9_ch7	10	1.0	1

In [19]:
df[(df['day_or_night'] == 'day') & (df['Channel'] == 'Monitor9_ch1')]

,Channel,Condition,Light_cycle,date,time,ZT_time,Dec_ZT_time,value,sleep_counts,bout,bout_length,day_or_night,bout_length_per_sleep_counts,channel_num
696,Monitor9_ch1,P384R,LD,2024-02-09,11:35:00,05:35:00,335,0,1,1,8,day,8.0,1
790,Monitor9_ch1,P384R,LD,2024-02-09,13:09:00,07:09:00,429,0,1,1,10,day,10.0,1
820,Monitor9_ch1,P384R,LD,2024-02-09,13:39:00,07:39:00,459,0,1,1,5,day,5.0,1
841,Monitor9_ch1,P384R,LD,2024-02-09,14:00:00,08:00:00,480,0,1,1,13,day,13.0,1
865,Monitor9_ch1,P384R,LD,2024-02-09,14:24:00,08:24:00,504,0,1,1,12,day,12.0,1
922,Monitor9_ch1,P384R,LD,2024-02-09,15:21:00,09:21:00,561,0,1,1,5,day,5.0,1
1064,Monitor9_ch1,P384R,LD,2024-02-09,17:43:00,11:43:00,703,0,1,1,5,day,5.0,1
2098,Monitor9_ch1,P384R,LD,2024-02-10,10:57:00,04:57:00,297,0,1,1,6,day,6.0,1
2287,Monitor9_ch1,P384R,LD,2024-02-10,14:06:00,08:06:00,486,0,1,1,7,day,7.0,1
2300,Monitor9_ch1,P384R,LD,2024-02-10,14:19:00,08:19:00,499,0,1,1,7,day,7.0,1
